In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# =========================
# CONFIG
# =========================
path = 'garbage_classification'   # your dataset path
labels = ['battery','biological','brown-glass','cardboard','clothes',
          'green-glass','metal','paper','plastic','shoes','trash','white-glass']
image_size = 128
extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif")

# =========================
# OPTIONAL: balance folders
# =========================
subfolder_images = {}
for root, dirs, files in os.walk(path):
    images = [os.path.join(root, f) for f in files if f.lower().endswith(extensions)]
    if images:
        subfolder_images[root] = images

if not subfolder_images:
    print("No images found. Check your path or extensions.")
else:
    min_count = min(len(imgs) for imgs in subfolder_images.values())
    print(f"Minimum images in a folder = {min_count}")
    for folder, images in subfolder_images.items():
        if len(images) > min_count:
            to_delete = random.sample(images, len(images) - min_count)
            print(f"Deleting {len(to_delete)} images from {folder} ...")
            for img in to_delete:
                os.remove(img)

# =========================
# IMAGE LOADING
# =========================
def adjust_brightness(img, alpha=1.5, beta=10):
    """Adjust brightness of image"""
    adjusted_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return adjusted_img

def load_images_from_directory(directory, label_list, image_size):
    images = []
    labels_arr = []
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    for label in label_list:
        folder_path = os.path.join(directory, label)
        for file_name in tqdm(os.listdir(folder_path), desc=f"Loading {label} images"):
            try:
                file_path = os.path.join(folder_path, file_name)
                img = cv2.imread(file_path)
                if img is not None:  # Check if image loaded
                    img = cv2.resize(img, (image_size, image_size))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # grayscale
                    img = clahe.apply(img)                      # CLAHE equalization
                    img = cv2.bitwise_not(img)                  # invert colors
                    img = adjust_brightness(img, alpha=1.5, beta=10)  # brightness

                    images.append(img)
                    labels_arr.append(label)
            except Exception as e:
                print(f"Error loading {file_name}: {e}")
    return images, labels_arr

# Load images
x, y = load_images_from_directory(path, labels, image_size)

# =========================
# TRAIN TEST SPLIT
# =========================
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

# Convert to numpy arrays
x_train = np.array(x_train).astype('float32')
x_test = np.array(x_test).astype('float32')

# Add channel dimension (N,128,128,1)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot
y_train = to_categorical([labels.index(label) for label in y_train], num_classes=len(labels))
y_test = to_categorical([labels.index(label) for label in y_test], num_classes=len(labels))

print(f"Train images: {x_train.shape}, Train labels: {y_train.shape}")
print(f"Test images: {x_test.shape}, Test labels: {y_test.shape}")

# =========================
# SHOW SAMPLE IMAGES
# =========================
colors_dark = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

fig, axes = plt.subplots(1, len(labels), figsize=(20, 20))
fig.text(s='Sample Image From Each Label', size=18, fontweight='bold',
         fontname='monospace', color=colors_dark[1], y=0.85, x=0.4, alpha=0.8)

for k, label in enumerate(labels):
    try:
        idx = np.where(np.argmax(y_train, axis=1) == k)[0][0]
        axes[k].imshow(x_train[idx].squeeze(), cmap='gray')
        axes[k].set_title(label.replace('_', ' ').title(), fontsize=14)
        axes[k].axis('off')
    except IndexError:
        print(f"No image found for label: {label}")
        axes[k].axis('off')
        axes[k].set_title(f"{label} (Not Found)", fontsize=14, color='red')

plt.tight_layout()
plt.show()

# =========================
# MODEL BUILDING
# =========================
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128,128,1)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))

model.summary()

# =========================
# COMPILE & TRAIN
# =========================
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    batch_size=32,
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)

